In [1]:
import networkx as nx
# import urllib -- Did not work on my linux machine - using requests instead
import re
import tqdm
import collections
import requests
#import community.community_louvain as cl
import json
import matplotlib.pyplot as plt
import pandas as pd
import copy

In [19]:

# to create the network I will create a simple function that gets references from wiki article

def get_references(text, pattern=r'\[\[(.*?)\]\]'):
    matches = re.findall(pattern, text)

    matches_more = []
    for match in matches:
        x = match.split('|')
        for m in x:
            matches_more.append(m)

    return matches_more

#The function above uses regex to gather everything inside double square brackets `[[]]`. We know that there might be more than one option there so after that we split everything on `|` mark.  



In [3]:

# Creating function to load up the rapper page

def get_wikipage(wiki_name):
    baseurl = "https://en.wikipedia.org/w/api.php?"
    action = "action=query"
    title = f"titles={wiki_name.replace(' ', '_')}" # replacing space with _
    content = "prop=revisions&rvprop=content"
    dataformat ="format=json" # we want have it in json format


    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)


    wikitext = requests.get(query).content.decode('utf-8') # decoding to utf

    d = json.loads(wikitext)

    # now we want the actual latest article from wikipedia,
    # so we have to access this information within dictionaries
    page = d['query']['pages']
    page_id = list(page.keys())[0]
    page_title = page[page_id]['title']
    content = page[page_id]['revisions'][0]['*']

    return page_title, content


In [4]:
page_title, content = get_wikipage('List_of_music_genres_and_styles')

In [5]:
content

"{{Short description|None}}\n{{Popular music}}\n\nThis is a '''list of music genres and styles'''. [[Music]] can be described in terms of many [[music genre|genres]] and styles. Classifications are often arbitrary, and may be disputed and closely related forms often overlap. Larger genres and styles comprise more specific sub-categories.\n\n==Classical==\n{{see also|List of classical and art music traditions}}\n* [[Andalusian classical music]]\n* [[Indian classical music]]\n* [[Korean court music]]\n* [[Persian classical music]]\n* [[Western classical music]]\n** [[Early music]] \n*** [[Medieval music]] (500–1400) \n**** [[Ars antiqua]] (1170–1310)\n**** [[Ars nova]] (1310–1377)\n**** [[Ars subtilior]] (1360–1420)\n****[[Bardcore]]\n****[[Neo-Medieval music]]\n****[[Tavernwave]]\n*** [[Renaissance music]] (1400–1600)\n***[[Baroque music]] (1600–1750)\n*** [[Galant music]] (1720–1770)\n***[[Classical period (music)|Classical period]] (1750–1820)\n***[[Romantic music]] (1780–1910) \n** [

In [6]:
music_list = get_references(content)

In [7]:
better_music_list = copy.copy(music_list)

In [8]:
for x in better_music_list:
    if x + ' (music)' in better_music_list:
        better_music_list.remove(x)
    elif x + ' music' in better_music_list:
        better_music_list.remove(x)
    elif x + ' in music' in better_music_list:
        better_music_list.remove(x)
    elif x + ' (genre)' in better_music_list:
        better_music_list.remove(x)



In [9]:
with open ('music_genres_new.txt', 'w') as f:
    for x in better_music_list:
        f.writelines(x + '\n')

UnicodeEncodeError: 'charmap' codec can't encode character '\u014d' in position 0: character maps to <undefined>

In [ ]:
# I then manually go through music genres and manually remove some of the values

In [10]:
with open ('music_genres.txt', 'w') as f:
    for x in better_music_list:
        f.writelines(x + '\n')

UnicodeEncodeError: 'charmap' codec can't encode character '\u014d' in position 0: character maps to <undefined>

In [25]:
_,wikipage_test = get_wikipage(music_list[5]) #cultural_origins[^|]*
test_text = get_references(wikipage_test, pattern=r'cultural_origins[^|]*')
test_text

['cultural_origins  = Early 600s – Mid-1400s, Korea\n']

In [69]:
def extract_origins(text):
    '''
    input is expected to be the raw output from get_references()
    '''
    start, end = 0,0
    matches_num = re.findall(r'\d+', text[0])
    
    if len(matches_num) == 2:
        # Contains two years, use these as start and end point
        start = int(matches_num[0])
        end = int(matches_num[1])
        
    elif len(matches_num) == 1:
        # Contains one year, likely decade, use start and end of this decade
        prefix = matches_num[0][:-1]
        start = int(prefix+"0")
        end = int(prefix+"9")
        
    elif len(matches_num) == 0:
        # No year given, write as unknown
        start, end = -1, -1
        
    else:
        raise Exception(f"UnhandledOriginException: the text \"{text[0]}\" contains more years than 2")
        
    place = re.findall(r',\s+\w+', text[0])[0][2:]
    
    return start, end, place
print(extract_origins(['cultural_origins  = Early 600s – Mid-1400s, Korea\n']))
print(extract_origins(['cultural_origins  = Early 600s – Mid-, Korea\n']))
print(extract_origins(['cultural_origins  = Early  – Mid-, Korea\n']))
print(extract_origins(['cultural_origins  = Early 600s – Mid-1400s sub 30s, Korea\n']))

(600, 1400, 'Korea')
(600, 609, 'Korea')
(-1, -1, 'Korea')


Exception: UnhandledOriginException: the text "cultural_origins  = Early 600s – Mid-1400s sub 30s, Korea
" contains more years than 2